In [ ]:
%pip install --upgrade google-cloud-speech

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
def transcribe_file(speech_file):
    """Transcribe the given audio file asynchronously."""
    from google.cloud import speech

    client = speech.SpeechClient()

    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    """
     Note that transcription is limited to 60 seconds audio.
     Use a GCS file for audio longer than 1 minute.
    """
    audio = speech.RecognitionAudio(content=content)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=32000,
        language_code="en-US",
        audio_channel_count=1,
    )

    # [START speech_python_migration_async_response
    operation = client.long_running_recognize(
        request={"config": config, "audio": audio}
    )
    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    response = operation.result(timeout=90)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.

    out = ""

    for result in response.results:
        # The first alternative is the most likely one for this portion.
        #print(u"Transcript: {}".format(result.alternatives[0].transcript))
        #print("Confidence: {}".format(result.alternatives[0].confidence))

        out += result.alternatives[0].transcript
        out += " "
    return out

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ""

In [ ]:
import os
files = os.listdir("gdrive")
files.sort()
files

In [ ]:
import io
path = "gdrive"
r = ""
for f in files:
 r += transcribe_file(os.path.join(path, f)) 

print(r)

In [ ]:
text_file = open("transcript.txt", "w")
text_file.write(r)
text_file.close()

In [ ]:
from google.colab import files
files.download("transcript.txt") 

In [ ]:
from google.colab import drive
drive.flush_and_unmount()